# Building Your First ML Pipeline
In this workshop we will work on a <strong>text classification</strong> task. The goal of text classification is to assign a label or a class to input text.

&nbsp;

 <<< *In case you are already familiar with the basics of text classification, you can skip the following intro and move on to __Dataset and model__ below it.* >>>
 
 &nbsp;


It is a very common task in <strong>Natural Language Processing (NLP)</strong> field and is quite straightforward to start with!


There are many uses cases for which text classification is used, some examples:

 
* Sentiment analysis and toxicity detection (review score prediction, comment section analysis)

* Topic classification (news article topic prediction, literature genre classification)


* Intent detection (user intent classification for Chatbots, search query intent)


* Spam detection, language identification etc.


Depending on a task, the classification can be either:
* Binary with only 2 classes

* Multi-class - multiple mutually exclusive classes

* Multi-label classification - not mutually exclusive, one entry can be assigned to none or to multiple labels

&nbsp;

 <<< *End of skippable info section.* >>>
 
 &nbsp;

## Dataset and model
We will be working on an <strong>intent detection</strong> task. 

Our dataset is called  [<strong>Banking77</strong>](https://huggingface.co/datasets/PolyAI/banking77), it contains online banking queries annotated with their corresponding intents. All queries are in English.
Overall, in this dataset there are 13,083 customer service queries labeled with 77 intents.

<strong>Our goal:</strong> build a model which can predict the intent for a previously unseen user query. Each query needs to be assigned to a single intent which makes this problem a multi-class text classification.


Predicting the intent of a user query is useful because it can make it possible to automate commonly needed functionality, save customers time and improve service satisfaction and ease load on customer support.

# What we will do:
* First, we will load the data and explore it


* Then, we will convert our texts to their vector representation to prepare the data for model training 


* We will try out 2 simple models and select the best parameters to use for them


* In the end, we will create a simple online API with Streamlit and test our solution live!

## Let's get started!

First, let's import what we will be using today. The needed libraries are:
* [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) - very commonly used tabular data analysis and manipulation tool
* [numpy](https://numpy.org/doc/stable/) -  also a very commonly used package for scientific computing in Python
* [scikit-learn](https://scikit-learn.org/stable/) - another library which is commonly used in data science and contains lots of tools for machine learning and statistical modeling 
* [sentence_transformers](https://sbert.net/) -  a library for accessing, using, and training embedding models, part of the Hugging Face ecosystem. We will use it to access one of the embeddings models to vectorize our texts.
* [datasets](https://huggingface.co/docs/datasets/en/index) - a library for easily accessing and sharing datasets, part of the Hugging Face ecosystem. We will only be using it to access the Banking77 dataset.
* [mlflow](https://mlflow.org/docs/latest/index.html) - an open-source platform for machine learning developments, most often used for model, artifact, parameter logging and sharing, experiment (model runs) comparison. 

They are already being defined and installed in the requirements.txt file.


In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset

import mlflow

from sentence_transformers import SentenceTransformer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import RandomizedSearchCV, cross_val_score

# Exploring the data

Loading the [dataset](https://huggingface.co/datasets/PolyAI/banking77):

In [ ]:
dataset = load_dataset("PolyAI/banking77", revision="main") # taking the data from the main branch

Our data is split in training and testing sets. Both have 2 columns: text ans label. Training data contains 10,003 rows and test data contains 3,080 rows. 

In [ ]:
dataset

You can read more about how to work with the datasets library [here](https://huggingface.co/docs/datasets/en/quickstart), but WE won't need anything other than the function above for this workshop.

Let's convert the datasets to pandas library DataFrame format: 

In [ ]:
train_data = pd.DataFrame(dataset['train'])
test_data = pd.DataFrame(dataset['test'])

Pandas makes working with data and doing simple data analysis very convenient. Below we will use some of the pandas functions like [head](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html), [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html), [describe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html), [plot](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html) for analysis.

First, let's look at the top rows of the dataset:

In [ ]:
train_data.head()

Our labels are numerical as you can see. These are positional indexes from the list of intent names (below) for this dataset. Let's add the labels names to make the data more interpretable. This will be useful since these labels are the intent which we want to predict.

In [ ]:
label_names = [
    "activate_my_card",
    "age_limit",
    "apple_pay_or_google_pay",
    "atm_support",
    "automatic_top_up",
    "balance_not_updated_after_bank_transfer",
    "balance_not_updated_after_cheque_or_cash_deposit",
    "beneficiary_not_allowed",
    "cancel_transfer",
    "card_about_to_expire",
    "card_acceptance",
    "card_arrival",
    "card_delivery_estimate",
    "card_linking",
    "card_not_working",
    "card_payment_fee_charged",
    "card_payment_not_recognised",
    "card_payment_wrong_exchange_rate",
    "card_swallowed",
    "cash_withdrawal_charge",
    "cash_withdrawal_not_recognised",
    "change_pin",
    "compromised_card",
    "contactless_not_working",
    "country_support",
    "declined_card_payment",
    "declined_cash_withdrawal",
    "declined_transfer",
    "direct_debit_payment_not_recognised",
    "disposable_card_limits",
    "edit_personal_details",
    "exchange_charge",
    "exchange_rate",
    "exchange_via_app",
    "extra_charge_on_statement",
    "failed_transfer",
    "fiat_currency_support",
    "get_disposable_virtual_card",
    "get_physical_card",
    "getting_spare_card",
    "getting_virtual_card",
    "lost_or_stolen_card",
    "lost_or_stolen_phone",
    "order_physical_card",
    "passcode_forgotten",
    "pending_card_payment",
    "pending_cash_withdrawal",
    "pending_top_up",
    "pending_transfer",
    "pin_blocked",
    "receiving_money",
    "Refund_not_showing_up",
    "request_refund",
    "reverted_card_payment?",
    "supported_cards_and_currencies",
    "terminate_account",
    "top_up_by_bank_transfer_charge",
    "top_up_by_card_charge",
    "top_up_by_cash_or_cheque",
    "top_up_failed",
    "top_up_limits",
    "top_up_reverted",
    "topping_up_by_card",
    "transaction_charged_twice",
    "transfer_fee_charged",
    "transfer_into_account",
    "transfer_not_received_by_recipient",
    "transfer_timing",
    "unable_to_verify_identity",
    "verify_my_identity",
    "verify_source_of_funds",
    "verify_top_up",
    "virtual_card_not_working",
    "visa_or_mastercard",
    "why_verify_identity",
    "wrong_amount_of_cash_received",
    "wrong_exchange_rate_for_cash_withdrawal",
]

In [ ]:
# assign label names from the list based on the index of the label 
train_data["label_name"] = train_data["label"].apply(lambda x: label_names[x])
test_data["label_name"] = test_data["label"].apply(lambda x: label_names[x])

Let's look at the top rows again. Now we have the label name as well:

In [ ]:
train_data.head()

Let's check the shape (# of row and columns of the data)

In [ ]:
print(
f"""
      Train dataset shape: {train_data.shape}, 
      Test dataset shape: {test_data.shape}
"""
)

Take a look at a random sample of 20 rows:

In [ ]:
train_data.sample(20)



To check the sample counts for the labels we have. We can use the pandas [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) function for this.


Looks like intent with the most labeled examples in the training data has 187 samples and is called "card_payment_fee_charged". The intent with the least amount of examples has 35 samples and is called "contactless_not_working".


In [ ]:
train_data["label_name"].value_counts()

Let's check the statistics on the count of labeled examples per intent with the [describe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) function below.


* We have 77 unique intents
* Mean number of examples per intent is 129, with standard deviation of 32
* The minimum is 35 examples and the maximum is 187
* The 25th percentile is 112 examples, median is 127, 75th percentile is 159


Seems like we have some deviations with less labeled intents, but overall the labels are relatively balanced.

In [ ]:
train_data["label_name"].value_counts().describe()

We can see the same distribution by using the *plot* function to create the bar plot of the sample count per label:

In [ ]:
train_data["label_name"].value_counts().plot.barh(title='Label Distribution', figsize=(10,15))

Let's check the lengths of the user query texts and the number of tokens (roughly words) per query: 

In [ ]:
train_data['text_length'] = train_data['text'].str.len()  # get the text length (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.len.html)
train_data['text_num_tokens'] = train_data['text'].str.split().str.len()  # split the text by spaces and get the length of the resulting list to calculate the number of tokens (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html)

Example of how we can get the length of the text and the number of tokens:

In [ ]:
text = "Here is a text"
tokens = text.split()

print(f"Length of the text is: {len(text)}")
print(f"Tokens of the text are: {tokens}")
print(f"The number of tokens is: {len(tokens)}")

We can use the [describe function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) from pandas to take a look at the statistics for user query text length:

In [ ]:
train_data['text_length'].describe()


## Task 1
Calculate the statistics for the number of tokens in user queries using the pandas [describe function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html). In addition to the default values, include the 10th and 90th percentiles.

In [ ]:
### YOUR CODE HERE ###


Once you have completed the previous step, let's plot the histograms of the distributions:

In [ ]:
train_data['text_length'].plot.hist(title='Text Length', bins=20)

In [ ]:
train_data['text_num_tokens']

In [ ]:
train_data['text_num_tokens'].plot.hist(title='Number of tokens', bins=20)

We can also take a lot of these distributions by label name to see if there are any significant differences. We can use the [groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function of pandas to calculate the statistics with the *describe* function per label:

In [ ]:
# Text length
text_length_statistics = train_data.groupby('label_name')['text_length'].describe()
text_length_statistics

### Task 2

Now calculate these statistics for the number of tokens columns, also using groupby by label name. Find which label has the highest mean number of tokens?

In [ ]:
### YOUR CODE HERE ###

Let's visualize the mean values for user query text length and token count per label:

In [ ]:
text_length_statistics.sort_values(by='mean')['mean'].plot.barh(title='Mean Text Length', figsize=(10,15))

## Task 3
Create a graph for the number of tokens per label like is done above for text lengths, but pick a different statistic other than the mean. Make sure the graph is sorted from highest to lowest value as well.

In [ ]:
### YOUR CODE HERE ###

text_length_statistics.sort_values(by='mean')['mean'].plot.barh(title='Mean Text Length', figsize=(10,15))

Looks like there are some differences between labels in terms of text length and token number. This can mean that they would be useful features for modeling.

## Text Embeddings

&nbsp;

 <<< *In case you are already familiair with the concept of text embeddings, you can skip the following intro.* >>>
 
 &nbsp;

To be able to train our model we need to convert our training data in the form of text into numerical representations. We also need to do the same for our test data to be able to make predictions. This conversion from text into numerical representations is also called vectorization.


There are several ways to vectorize text data, one of them which is often used is text embeddings. Text embeddings are vectors (lists) or floating point numbers and they are designed to capture the semantic meaning and context of the words they represent. There are many models available which can be used for getting embeddings from given text. Some examples, from simple to closer to state-of-the-art, are:
* [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html)
* [FastText](https://fasttext.cc/docs/en/crawl-vectors.html)
* [Universal sentence encoder (USE)](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder.ipynb)
* [BERT](https://huggingface.co/docs/transformers/en/model_doc/bert)
* [DistilBERT](https://huggingface.co/docs/transformers/en/model_doc/distilbert)
* [XLNet](https://huggingface.co/docs/transformers/en/model_doc/xlnet)
* [MPNet](https://huggingface.co/docs/transformers/en/model_doc/mpnet)
* [GPT](https://platform.openai.com/docs/guides/embeddings)




You can check more available models on, for example but not limited to, [sentence_transformers](https://sbert.net/docs/sentence_transformer/pretrained_models.html) and Hugging Face [model hub](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) (sentence similarity models linked), there are also others. 


Embeddings are useful in multiple kinds of tasks:
* Text similarity measurement
* Text classification
* Semantic search
* Text clusterization
* Recommendations


and others.


Embeddings can also be limited to a single language or be multilingual and support a varying number of languages.

&nbsp;

 <<< *End of skippable intro on text embeddings* >>>
 
 &nbsp;

For building our model, we will make use of the idea that very similar sentences likely belong to the same intent label.
For our embeddings, we will take an MPNet English language model developed by Microsoft and called [all-mpnet-base-v2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2). It was trained for sentence similarity tasks and therefore produces embeddings which work well for our needs. 

Example of how the embeddings can be accessed, we need to load the model with needed path and use the *encode* function. See [here](https://huggingface.co/sentence-transformers/all-mpnet-base-v2):

In [ ]:
sentences = ["This is an example sentence", "Each sentence is converted"]

embeddings_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = embeddings_model.encode(sentences)
print(embeddings)

Let's check on one of the queries from our data:

In [ ]:
embeddings_model.encode('I am still waiting on my card?')

In [ ]:
# Let's check the dimentions of the produced vector
embeddings_model.encode('I am still waiting on my card?').shape

## Task 4
Semantic similarity between texts is often measured with cosine similarity. Mathematically, it's the cosine of the angle between 2 vectors, meaning the dot product of the vectors divided by the product of their lengths. When we want to calculate the cosine similarity of 2 texts, text embeddings can be used as the vector representation.


Your task is to <strong>calculate the cosine similarity of 2 given texts</strong> using the all-mpnet-base-v2 embeddings model we downloaded above. You can implement the cosine similarity function yourself using numpy (or look up how to implement it in Python).
 
Feel free to try out any of your own text examples!

In [ ]:

text_1 = "When will my card be delivered?"	
text_2 = "How long will my card delivery take?"

### YOUR CODE HERE ###


Once we are done with task 4, let's proceed to training our model! But first we need to vectorize our train and test datasets using the embeddings model.

## Preparing the data

To be able to train and test our model we need to convert all our user queries in the training and test datasets into embeddings.

We can do so by putting all of the queries of respective datasets into lists with a *tolist* function and then applying the embedding model the same way as in the examples above. This will create 2 embedding matrices which are ready for model training and inference.

In [ ]:
train_text_lists = train_data["text"].tolist()
test_text_lists = test_data["text"].tolist()

In [ ]:
# command takes around 4-5 mins to run
train_embeddings = embeddings_model.encode(train_text_lists, show_progress_bar=True)
test_embeddings = embeddings_model.encode(test_text_lists, show_progress_bar=True)

# Modeling

### Algorithms

&nbsp;

 <<< *In case you are already familiair with the basics of k-NN and/or Logistic Regression, you can skip the following two info sections and move on to the __K-nearest Neighbors__ section below them.* >>>
 
 &nbsp;

We will train 2 simple models based on [K-nearest Neighbors](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) and [Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression) algorithms.

<strong>k-Nearest Neighbors (k-NN) Classification</strong>
k-NN classification works by memorizing all the training examples and then classifying a new data point based on its proximity to these memorized examples. For prediction, the algorithm calculates the distance between the new data point and examples in the training set. It then identifies the k closest training examples, i.e. neighbors, and assigns to the new data point the class that is most common among these k neighbors. All neighbors can either count equally to the predicted class vote or the closer neighbors can have more weight. The final predictions are scores representing the class vote distribution.

<strong>Logistic Regression</strong>
Logistic regression operates by first computing a linear combination of the input features. The result is then passed through the logistic (sigmoid) function, which maps it to a probability between 0 and 1. This probability represents the likelihood of the data point belonging to the positive class. If the probability is above a certain threshold (typically 0.5 but this can be tuned depending on the needs) the data point is classified as belonging to the positive class. Otherwise, it is classified as belonging to the negative class. Logistic regression can also be extended to multi-class classification.

&nbsp;

 <<< *End of skippable info sections.* >>>
 
 &nbsp;

## K-nearest Neighbors

Let's start by fitting a K-nearest Neighbors model on the training data.
We will use scikit-learn implementation: [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier).

Although there are a lot of details, the scikit-learn API is fairly simple to use in practice: each estimator has a `.fit(X, y)` method to train a model and a `.predict(X)` method to predict on unseen data.

We will start by fitting the model with its default parameters and this will give us a baseline model, upon which we can improve.

In [ ]:
# Initialize KNeighborsClassifier and fit on training data
model_baseline = KNeighborsClassifier()
model_baseline.fit(train_embeddings, train_data['label_name'])

In [ ]:
# Let's check the results on cross-validation (more on how cross-validation works in Model Evaluation section)
cv_scores = cross_val_score(estimator=model_baseline, X=train_embeddings, y=train_data['label_name'], scoring="f1_macro", cv=3)

print(
    f"""
      Baseline model CV scores by fold: {cv_scores}, 
      Mean CV score {cv_scores.mean()}
"""
)


# Hyperparameters Tunning

The process of finding the optimal hyperparameters for your task is called [hyperparameter tuning](https://scikit-learn.org/stable/modules/grid_search.html). It involves experimenting with different hyperparameter values of an algorithm to find the combination that yields the best model performance on a validation set. This can be done using techniques like [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV), where all possible combinations are tried, or random search, where random combinations are tested.

To tune hyperparameters for our models we will use [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). It optimizes hyperparameters by randomly sampling a specified number of combinations from a given range. Each combination is evaluated using [cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html), and the best performing set is selected. This method is more efficient than exhaustive searches, as it explores the hyperparameter space more quickly while still finding effective parameter settings.

### K-nearest Neighbors Hyperparameter tuning

Below we define values for our hyperparameter choices, refer to the scikit-learn documentation for more details on [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) settings.

We are using the F1-score metric since it is well suited for our task - customer support text classification. For this case, it's important to balance precision and recall which [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score) helps to achieve. "Macro" in F1-macro is an averaging strategy, it means that the metrics are calculated for each label separately, and then their unweighted mean is found for the final score.

In [ ]:
params = {
    "n_neighbors": [3, 5, 7, 9, 11, 13, 15], # number of neighbors
    "weights": ["distance", "uniform"], # whether the votes from all neighbors should be counted equally or by distance to the prediction point
    "metric": ["cosine", "euclidean" ], # which metric to use for distance calculation
}

# define RandomizedSearchCV
clf = RandomizedSearchCV(
    estimator=KNeighborsClassifier(n_jobs=-1),
    param_distributions=params,
    n_iter=20,
    random_state=0,
    cv=3,
    scoring="f1_macro",
)

In [ ]:
# Run the hyperparameter tunining given the train data (train_embeddings) and target values (train_data['label_name']), command takes around 40s to run
search = clf.fit(train_embeddings, train_data['label_name']) 

Take a look at the best hyperparameters found and the achieved F1-score on cross-validation:

In [ ]:
print(f"""
      Best parameters: {search.best_params_}
      F1-score: {search.best_score_}
""")

In [ ]:
# Let's check the cross-validation score by fold for the model with tuned hyperparameters:

best_model = search.best_estimator_
cv_scores_tuned = cross_val_score(estimator=best_model, X=train_embeddings, y=train_data['label_name'], scoring="f1_macro", cv=3)

print(
    f"""
      Best model CV scores by fold: {cv_scores_tuned}, 
      Mean CV scores: {cv_scores_tuned.mean()} # same as above for search.best_score_
"""
)

We can see that, although very slightly, the f1_score on cross validation has increased after tuning hyperparameters. Depending on the task and data this difference can be more dramatic so tuning hyperparameters is worth the effort.

## Task 5

Tune hyperparameters for a Logistic regression model via RandomizedSearchCV like in the example above. Consider the following hyperparameters: `C`, `penalty` and `solver`, but you can also add any other ones you want to tune! Refer to [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression) documentation for hyperparameter choices.

<strong>Hint:</strong> for the values of C you can define a disrtibution of values to check with [numpy](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) like this: `np.linspace(start=0.001, stop=10, num=21)`

In [ ]:

distributions = {
    'C': , ### YOUR CODE HERE ### 
    'penalty': ### YOUR CODE HERE ### 
    'solver': ### YOUR CODE HERE ###
}

# Hint: use a higher number of iterations (n_iter=100) for RandomizedSearchCV since more combinations would be possible here than in k-NN example
log_clf = ### YOUR CODE HERE ###

In [ ]:
log_search = log_clf.fit(train_embeddings, train_data['label_name']) 

In [ ]:
print(f"""
      Best parameters: {log_search.best_params_}
      F1-score: {log_search.best_score_}
""")

# Model Evaluation

&nbsp;

 <<< *In case you are already familiair with the different components of model evaluation and cross-validation, feel free to skip the next info section.*>>>
 
 &nbsp;
 
### Components of [Model Evaluation](https://scikit-learn.org/stable/model_selection.html):

* [**Metric choice**](https://scikit-learn.org/stable/modules/model_evaluation.html): consider what is suitable for your task and model, as well as, what best reflects the requirements for the solution.
  * For example, in a customer support chatbot, we might care about [precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) equally and therefore F1 would be the optimal metric. Or it might be that precision is more important since we would rather transfer the request to a human when the bot is unsure. Then we could use a weighted F1 where precision is weighed higher.
  * In some cases of binary classification, for example, it could be that we care about only the top N results with the highest prediction score, therefore we need to compute our metrics on this top set (it's called @k, for example, precision@k).
  * Also, in some cases of binary classification, it's important that your true positive examples get a higher predicted score, then [ROC AUC score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) is a suitable metric. In other cases, it might not be important and then accuracy, F1, precision, and recall could work better.
  * Don't use [accuracy score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) for imbalanced datasets since if your positive class is only 5% of the data, predicting all to be negative will give a 95% accuracy which seems high but is useless. Precision, recall, F1 would be better choices.  

* **Validation set evaluation**: This is also a possibility, especially for non-standard cases when cross-validation is challenging to apply. Be careful not to overfit the validation set! Note that the validation set is different from a test set in this case. The data is split into 3 parts - training, validation, and testing.

* **Test set evaluation**:
  * The test set should not be used during data preprocessing, model training, or hyperparameter tuning to prevent data leakage and biasing the evaluation results.
  * The purpose is to provide a final, unbiased evaluation of the model's performance.
  * Be careful to make sure that the test set is representative of the real production data and that the way it was split is suitable for your task. The same considerations as for cross-validation exist here; time series data or not independently and identically distributed data need to be treated differently.
  * Take note of the class imbalances in the test set compared to the production data. This can affect your metrics calculation reliability.
  * Final evaluation is always best in the production setting (such as in an A/B test), since only that can show the real performance and impact of the model

* [**Cross-validation**](https://scikit-learn.org/stable/modules/cross_validation.html):
  * Cross-validation works by splitting the training data into multiple subsets (folds). The model is trained on all of these subsets but one and is evaluated on this remaining one. This process is repeated several times (same as the number of folds), and the results are averaged.
  * It is often used for hyperparameter tuning and model selection. By using multiple folds, it generally provides a more robust estimate of the model's performance compared to a single train-test split.
  * Be careful with using cross-validation for time series data or data that is not independently and identically distributed (one row is not an independent entry, for example, the same user_id is present in the data multiple times). Use evaluation approaches suitable for your kind of task, such as [time series-aware validation splitting](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-of-time-series-data) or [Grouped cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html#group-cv).

 &nbsp;

 <<< *End of skippable info section.*>>>
 
 &nbsp;


Let's train our k-NN model with the hyperparameters which we found in the previous step with RandomizedSearchCV:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5, weights='distance', metric='cosine')
knn.fit(train_embeddings,  train_data['label_name'])

Getting the predictions on the test set to measure the final metrics:

In [ ]:
prediction_knn = knn.predict(test_embeddings)

In our task of a customer support text classification, the metrics which are of interest to us are precision, recall and F1-score. We can use [classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) function to calculate them for all classes separately and also get the overall results.

In [ ]:
print(classification_report(test_data['label_name'], prediction_knn))

In [ ]:
clf_report_dict = classification_report(test_data['label_name'], prediction_knn, output_dict=True) # get the same results in a dict format 

In [ ]:
clf_report_dict['macro avg']

## Task 6
1. Train a Logistic Regression model using the hyperparameters found in Task 5.

2. Get predictions on the test data
3. Calculate the metrics using classification_report function, as in the example above for k-NN

In [ ]:
### YOUR CODE HERE ###

## Check your own query
Once you are done with task 6, let's check our model on any new queries and output top 3 predicted classes with the highest scores. To access predicted scores we can use .predict_proba() instead of .predict()

In [ ]:
def get_top_k_predictions(query, embed_model, clf_model, k):
    # Function to output top k predicted classes with the highest scores
    query_embedding = embed_model.encode(query).reshape(1, -1)
    prediction_scores = filter(
            lambda pr: pr[1] > 0,
            zip(clf_model.classes_, clf_model.predict_proba(query_embedding)[0]),
        ) # keep non 0 scores

    prediction_result = sorted(prediction_scores, key=lambda x: x[1], reverse=True)[:k]

    return prediction_result

In [ ]:
test_query = "I still haven't recieved my card, when will it be ready?" # feel free to change to your own query!
get_top_k_predictions(query=test_query, embed_model=embeddings_model, clf_model=knn, k=3)

In [ ]:
# let's try a more difficult one
test_query = "I don't see my transactions reflected in my account balance. How can I fix it?"
get_top_k_predictions(query=test_query, embed_model=embeddings_model, clf_model=knn, k=3)

# Model Versioning with MLFLow

Model versioning refers to save the model for later usage. 
Often is the case we want to separate the process of building the model from that of using the model for production.
MLFLow is an open-source solution to do model versioning. Beydond saving the model, you can compare different versions of the model via an UI, add labels to the models (like production, testing, etc).

In [ ]:
mlflow.set_tracking_uri("file:///workspaces/build-your-first-ml-pipeline-workshop/mlruns")
mlflow.sklearn.log_model(knn, 'model')
mlflow.log_metrics(clf_report_dict['macro avg'])
mlflow.log_params(clf.best_params_)

# Task 7

Load the saved model back to the notebook using its mlflow id.

You can find the mlflow run id in `/notebooks/mlruns/0/<run_id>/..`

In [ ]:
run_id = ### YOUR CODE HERE ###
model = mlflow.sklearn.load_model(f"runs:/{run_id}/model")
model.predict(embeddings_model.encode("I still haven't recieved my card, when will it be ready?").reshape(1, -1))

# Next steps: Creating the pipeline

We finished the notebook work! Congratulations to reaching this point!

Now go to: notebooks/test_pipeline.ipynb and fill in the missing lines in the pipeline

# Bonus Improvements tasks 

What you can do if you want to further improve your model:
* [Tune the decision threshold to improve precision](https://scikit-learn.org/stable/modules/classification_threshold.html#)
* Conduct error analysis - on a validation set (not test set!) check on which examples your model is making mistakes, any patterns you can draw from it which could be fixed by additional features, fixing labeling mistakes, tuning decision threshold?
* Train a more complex model, to learn how to fine-tune a transformer model for text classification you can check these resources: Hugging Face [article](https://huggingface.co/docs/transformers/v4.17.0/en/tasks/sequence_classification) and [example notebook](https://github.com/huggingface/notebooks/blob/main/examples/text_classification.ipynb)